## 필요 데이터 전처리
1.  두 데이터의 수집 속도가 다름(128hz vs 100hz)이를 학습하려면 동기화가 필요
    -   업셈플링(3200hz)을 진행하면 데이터의 손실 없이 가능 하지만 연산량이 너무 많아짐
    -   100hz 다운샘플링: tdcsfog에서 약간의 손실이 있지만 대부분의 데이터에서 손실없이 사용 가능
    
2.  사람이 분석한 라밸링은 오차가 들어갈 수 있고 라밸 경계의 모호함을 표현할 수 있어, 하드라밸링이 아닌 스무딩을 통해 소프트 라밸링으로 변환
    -   다운 샘플링을 통해 희석되는 fog 라밸링 또한 소프트 라밸링을 통해 유지 가능
3.  절대 이동량, 상대 이동량, 각 방향의 델타, 이동량 델타 추가: 단순한 각 축의 가속도 뿐 아니라 이동량, 변화량(델타)을 변수로 생성
4.  변화량과 이동량을 텐서에 넣기 or 텐서 기준으로 
5.  텐서 변환: 시계열의 특성을 반영과 딥러닝을 위하여 텐서로 변환

##### 파일 로드

In [21]:
import pandas as pd
test_df = pd.read_csv(r'train\tdcsfog\0a89f859b5.csv')

### 소프트 라벨링
-   라벨링 방식: 연구자들이 환자의 비디오를 보고 FOG의 시작/종료 시점을 근거로 라밸링을 부여함
-   경계 불확실: 사람의 판단에는 불가피한 휴먼 에러가 있어, 실제 지점과 주석 지점에 오차가 있을 수 있음
-   소프트 라벨링: 시작/종료 경계구간만 완만히 스무딩 하고 내부는 하드 라벨링 유지
-   목표: 모델이 경계의 불확실성을 학습하여 경계 깜빡임을 줄이고 성능을 높이는것을 목표함

#### 소프트 라벨링 적용

$$
y(t)=
\begin{cases}
0, & t < s-\Delta, \\[4pt]
\displaystyle \tfrac{1}{2}\!\left(1 - \cos\!\frac{\pi\,(t - s + \Delta)}{2\Delta}\right), 
& s-\Delta \le t \le s+\Delta, \\[8pt]
1, & s+\Delta < t < e-\Delta, \\[4pt]
\displaystyle \tfrac{1}{2}\!\left(1 + \cos\!\frac{\pi\,(t - e + \Delta)}{2\Delta}\right), 
& e-\Delta \le t \le e+\Delta, \\[8pt]
0, & t > e+\Delta.
\end{cases}
$$
t: 시간, s: 이벤트 시작 시간, e: 이벤트 종료 시간, Δ: 스무딩 구간 길이

-   구간: 0.3, 0.2, 0.1(s)스무딩 적용
-   방식: cos lapping 진행


In [12]:
import numpy as np
import ruptures as rpt

In [ ]:
def search_boundary(a):
    a = np.array(a)
    point = np.flatnonzero(np.diff(a)) + 1
    starts = point[a[point] == 1].astype(int).tolist()
    ends = point[a[point] == 0].astype(int).tolist()
    return starts, ends

In [ ]:
def smoding(df, col, start_idxs, end_idxs):
    

In [36]:
a, b = search_boundary(test_df['Turn'])

In [ ]:
def cosin_lapping_and_sampling(df, trens_hz=100, interval=0.2, folder="tdcsfog"):
    label_cols = ['StartHesitation','Turn','Walking', 'Event']
    if folder in ["tdcsfog", "defog"]:
        start = df[label_cols[0]]
        turn = df[label_cols[1]]
        walk = df[label_cols[2]]
        cols = [start, turn, walk]
        num = 3
    elif folder == "notype":
        event = np.array(df[label_cols[3]])
        cols = [event]
        num = 1
    else:
        print('라벨이 없습니다.')
        return

    if folder == 'tdcsfog':
        hz = 128
    else:
        hz = 100

    index_list = []
    for col in cols:
        s, e = search_boundary(col)
        index_list.append((s,e))








    return index_list
        


In [46]:
sam_list = cosin_lapping_and_sampling(test_df)
sam_list

[([], []), ([2678], [3126]), ([], [])]

In [45]:
test2_df = pd.read_csv(r'train\tdcsfog\fd52a32229.csv')

In [47]:
sam_list = cosin_lapping_and_sampling(test2_df)
sam_list

[([], []), ([4038, 8013], [4995, 8144]), ([], [])]